In [20]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from pymongo import MongoClient
import requests
%load_ext autoreload
%autoreload 2
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from pymongo import MongoClient
import requests
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


mc = MongoClient()
capstone_db = mc['bit_scraping']
#artist_names  = capstone_db['artist_names']
artist_info = capstone_db['artist']

In [21]:
#gd.download_band_names(['OR'],'bands_in_ca.csv')

In [22]:
#gd.download_band_names(['OR','CA'],'bands_in_ca.csv')

In [23]:
#artist_info.delete_many({})

In [30]:
data = pd.read_csv('venues.csv')

In [31]:
data = data[data.region.isin(['WA','OR','CA'])]

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10796 entries, 4 to 42340
Data columns (total 11 columns):
Unnamed: 0        10796 non-null int64
venueidentifer    10796 non-null object
venue_name        10796 non-null object
country           10796 non-null object
region            10796 non-null object
city              10796 non-null object
genre             10796 non-null object
bio               10796 non-null object
latitude          10737 non-null float64
longitude         10737 non-null float64
venue_id          10796 non-null int64
dtypes: float64(2), int64(2), object(7)
memory usage: 1012.1+ KB


In [33]:
seattle = data[data.city == 'Seattle']

In [38]:
seattle.sort_values(by = 'venue_name', axis = 0)[['venue_name','latitude', 'longitude']]

,venue_name,latitude,longitude
138,Showbox Theatre,47.606209,-122.332071
200,"""Capsize"" Record Release Show",47.606209,-122.332071
205,"""Pick a Number"" Video Premier (plus Coco!)",47.448930,-122.282916
265,'On the Dock' Concert Series at The College Cl...,47.606389,-122.330833
355,100.7 The Wolf Throwdown 2018,47.606209,-122.332071
367,107.7 The End Summer Camp,47.606389,-122.330833
392,12th Annual Tom Waits Tribute Night,47.606389,-122.330833
394,12th Ave Arts,47.606389,-122.330833
393,12th Ave Arts - Studio Theatre,47.606389,-122.330833
397,13 Coins,47.606389,-122.330833


In [9]:
df['event_counts'] = 1

In [1]:
df.head(5)

NameError: name 'df' is not defined

In [11]:
ratings_matrix = df.groupby(['artists_id', 'venue_id'], as_index = False)

In [12]:
dt = pd.DataFrame(ratings_matrix.count())

In [13]:
dt.event_counts = 1

In [14]:
dt.to_csv('test_ratings_matrix.csv')

In [15]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [16]:
#sp_df = sc.read.format("csv").option("header", "true").load("test_ratings_matrix.csv.csv")

In [17]:
sp_df = spark.createDataFrame(dt)

In [18]:
sp_df.show()sp_df = spark.createDataFrame(dt)

+----------+--------+------------+
|artists_id|venue_id|event_counts|
+----------+--------+------------+
|         4|    2145|           1|
|         7|   15279|           1|
|         7|   29315|           1|
|        12|   12015|           1|
|        12|   21217|           1|
|        12|   23893|           1|
|        12|   38693|           1|
|        12|   40553|           1|
|        14|   13461|           1|
|        14|   15279|           1|
|        14|   22147|           1|
|        14|   24747|           1|
|        14|   32170|           1|
|        14|   33537|           1|
|        16|    7144|           1|
|        16|   14657|           1|
|        16|   15628|           1|
|        16|   21086|           1|
|        16|   31141|           1|
|        16|   37114|           1|
+----------+--------+------------+
only showing top 20 rows



In [19]:
train, test = sp_df.randomSplit([0.8, 0.2], seed=42)

In [27]:
als_model = ALS(userCol='venue_id',
                itemCol='artists_id',
                ratingCol='event_counts',
                nonnegative=True,
                regParam=0.1,
                rank=5,
                implicitPrefs=True,
                alpha = 40)

In [28]:
recomender = als_model.fit(train)

In [29]:
rjj = 11270709

In [30]:
venues_in_seattle = np.unique(data[data['city'] == 'Seattle']['venue_id'])

In [31]:
rate_this = pd.DataFrame()

In [32]:
rate_this['venue_id'] = venues_in_seattle

In [33]:
rate_this['artists_id'] = rjj

In [34]:
#rate_this.info()

In [35]:
rate_this = rate_this[['artists_id', 'venue_id']]

In [36]:
rate_this_spark = spark.createDataFrame(rate_this)

In [37]:
#rate_this_spark.show()

In [38]:
predictions = recomender.transform(rate_this_spark)

In [39]:
#predictions.show()

In [40]:
venues = pd.read_csv('venues.csv')

In [41]:
thing = predictions.dropna().sort('prediction',ascending=False)

In [42]:
predictionspd = thing.toPandas()

In [43]:
top_pridictions = predictionspd['venue_id'].values[0:10]

In [44]:
venues[venues['venue_id']== 4239]

,Unnamed: 0,venueidentifer,venue_name,country,region,city,genre,bio,latitude,longitude,venue_id
4239,4239,Black Lodge Seattle WA United States,Black Lodge,United States,WA,Seattle,"['Space Rock, Alternative, Post Hardcore']",['When great bands break up there is rarely an...,47.606209,-122.332071,4239


In [45]:
np.unique(data[data['venue_id'].isin(top_pridictions)]['venue_name'])

array(['Bumbershoot Festival', 'CenturyLink Field',
       'Foundation Nightclub', 'Highline', 'Moore Theatre',
       'Neptune Theatre', 'Paramount Theatre', 'Showbox SODO',
       'Showbox SoDo', 'The Showbox'], dtype=object)

In [46]:
data[data['artist_name']== 'Royal Jelly Jive']

,Unnamed: 0,_id,artist_facebook,artist_image,artist_name,artist_thumb,artist_url,artists_id,bio,genre,...,event_url,band_name,venue_name,country,region,city,latitude,longitude,venueidentifer,venue_id
96997,97001,5b785c10482deb635586e9fd,NaN,https://s3.amazonaws.com/bit-photos/large/2593...,Royal Jelly Jive,https://s3.amazonaws.com/bit-photos/thumb/2593...,https://www.bandsintown.com/a/11270709?came_fr...,11270709,Few bands have the ability to evoke an old-sch...,"Rock, Soul, R&b/soul, Swing And Hip-hop, Rnb-soul",...,https://www.bandsintown.com/e/100198192?app_id...,Royal Jelly Jive,Francis Ford Coppola Winery,United States,CA,Geyserville,38.707476,-122.902765,Francis Ford Coppola Winery Geyserville CA Uni...,12452
96998,97002,5b785c10482deb635586e9fd,NaN,https://s3.amazonaws.com/bit-photos/large/2593...,Royal Jelly Jive,https://s3.amazonaws.com/bit-photos/thumb/2593...,https://www.bandsintown.com/a/11270709?came_fr...,11270709,Few bands have the ability to evoke an old-sch...,"Rock, Soul, R&b/soul, Swing And Hip-hop, Rnb-soul",...,https://www.bandsintown.com/e/100198206?app_id...,Royal Jelly Jive,Healdsburg Plaza,United States,CA,Healdsburg,38.610466,-122.869161,Healdsburg Plaza Healdsburg CA United States,14691
96999,97003,5b785c10482deb635586e9fd,NaN,https://s3.amazonaws.com/bit-photos/large/2593...,Royal Jelly Jive,https://s3.amazonaws.com/bit-photos/thumb/2593...,https://www.bandsintown.com/a/11270709?came_fr...,11270709,Few bands have the ability to evoke an old-sch...,"Rock, Soul, R&b/soul, Swing And Hip-hop, Rnb-soul",...,https://www.bandsintown.com/e/1010352316?app_i...,Royal Jelly Jive,Miners Foundry Cultural Center,United States,CA,Nevada City,39.262006,-121.020402,Miners Foundry Cultural Center Nevada City CA ...,22002
97000,97004,5b785c10482deb635586e9fd,NaN,https://s3.amazonaws.com/bit-photos/large/2593...,Royal Jelly Jive,https://s3.amazonaws.com/bit-photos/thumb/2593...,https://www.bandsintown.com/a/11270709?came_fr...,11270709,Few bands have the ability to evoke an old-sch...,"Rock, Soul, R&b/soul, Swing And Hip-hop, Rnb-soul",...,https://www.bandsintown.com/e/1010079256?app_i...,Royal Jelly Jive,Yoshi's,United States,CA,Oakland,37.794665,-122.277178,Yoshi's Oakland CA United States,41691
97001,97005,5b785c10482deb635586e9fd,NaN,https://s3.amazonaws.com/bit-photos/large/2593...,Royal Jelly Jive,https://s3.amazonaws.com/bit-photos/thumb/2593...,https://www.bandsintown.com/a/11270709?came_fr...,11270709,Few bands have the ability to evoke an old-sch...,"Rock, Soul, R&b/soul, Swing And Hip-hop, Rnb-soul",...,https://www.bandsintown.com/e/100198237?app_id...,Royal Jelly Jive,Capitola Art and Wine Festival,United States,CA,Capitola,36.975228,-121.953293,Capitola Art and Wine Festival Capitola CA Uni...,6314
97002,97006,5b785c10482deb635586e9fd,NaN,https://s3.amazonaws.com/bit-photos/large/2593...,Royal Jelly Jive,https://s3.amazonaws.com/bit-photos/thumb/2593...,https://www.bandsintown.com/a/11270709?came_fr...,11270709,Few bands have the ability to evoke an old-sch...,"Rock, Soul, R&b/soul, Swing And Hip-hop, Rnb-soul",...,https://www.bandsintown.com/e/1008505774?app_i...,Royal Jelly Jive,Harvest Fest w/Avett Brothers & Lake Street Dive,United States,CA,Glen Ellen,38.364167,-122.523056,Harvest Fest w/Avett Brothers & Lake Street Di...,14582
97003,97007,5b785c10482deb635586e9fd,NaN,https://s3.amazonaws.com/bit-photos/large/2593...,Royal Jelly Jive,https://s3.amazonaws.com/bit-photos/thumb/2593...,https://www.bandsintown.com/a/11270709?came_fr...,11270709,Few bands have the ability to evoke an old-sch...,"Rock, Soul, R&b/soul, Swing And Hip-hop, Rnb-soul",...,https://www.bandsintown.com/e/100198262?app_id...,Royal Jelly Jive,Lost Sierra Hoedown,United States,CA,Blairsden-graeagle,39.766760,-120.698070,Lost Sierra Hoedown Blairsden-graeagle CA Unit...,20210
97004,97008,5b785c10482deb635586e9fd,NaN,https://s3.amazonaws.com/bit-photos/large/2593...,Royal Jelly Jive,https://s3.amazonaws.com/bit-photos/thumb/2593...,http